In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import datetime

def readData():
    
    global x_scaled_training, y_scaled_training, x_scaled_testing, y_scaled_testing, x_scaler, y_scaler, x_train, x_test, y_train, y_test
    
    dataFrame = pd.read_csv("house_data.csv")
    
    del dataFrame["house_number"]
    del dataFrame['street_name']
    del dataFrame['unit_number']
    del dataFrame['zip_code']

    featuresDataFrame = pd.get_dummies(dataFrame, columns=["city", "garage_type"])
    
    del featuresDataFrame['sale_price']
    
    global x_train, x_test, y_train, y_test
    
    x=featuresDataFrame.as_matrix()
    y=dataFrame[['sale_price']].as_matrix()
    
    x_scaler = MinMaxScaler(feature_range=(0, 1))
    y_scaler = MinMaxScaler(feature_range=(0, 1))
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
    
    x_scaled_training = x_scaler.fit_transform(x_train)
    y_scaled_training = y_scaler.fit_transform(y_train)
    
    x_scaled_testing = x_scaler.transform(x_test)
    y_scaled_testing = y_scaler.transform(y_test)
    

def loadModel():
    with tf.Session() as session:
        saver=tf.train.import_meta_graph('./Models/saved_model.ckpy.meta')
        saver.restore(session, tf.train.latest_checkpoint('./Models/'))
        
        graph=tf.get_default_graph()
        
        output=graph.get_tensor_by_name("output/output:0")
        x= graph.get_tensor_by_name('input/x:0')
        
         # Now that the neural network is trained, let's use it to make predictions for our test data.
        # Pass in the X testing data and run the "prediciton" operation
        y_predicted_scaled = session.run(output, feed_dict={x: x_scaled_testing})
    
        # Unscale the data back to it's original units (dollars)
        y_predicted = y_scaler.inverse_transform(y_predicted_scaled)
    
        house_real_pricing = y_test[100:105]
        predicted_pricing = y_predicted[100:105]
    
        print("The actual house price of House_1 was $\n{}".format(house_real_pricing))
        print("Our neural network predicted prices of $\n{}".format(predicted_pricing))

readData()        
loadModel()

/Users/milse113/anaconda3/envs/AI/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/milse113/anaconda3/envs/AI/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/milse113/anaconda3/envs/AI/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


INFO:tensorflow:Restoring parameters from ./Models/saved_model.ckpy
The actual house price of House_1 was $
[[396897.]
 [793799.]
 [447302.]
 [139857.]
 [214198.]]
Our neural network predicted prices of $
[[385902.4  ]
 [816438.94 ]
 [373526.44 ]
 [ 55517.203]
 [ 69143.18 ]]
